In [1]:
from smarty.forcefield import *
import parmed
import openeye.oechem as oechem
from simtk.openmm.app import PDBFile

In [9]:
pdbfile = PDBFile('cyclohexane_ethanol_0.4_0.6.pdb')
ff = ForceField('../../smarty/data/forcefield/Frosst_AlkEtOH.ffxml')

mols = []
flavor = oechem.OEIFlavor_Generic_Default | oechem.OEIFlavor_MOL2_Default | oechem.OEIFlavor_MOL2_Forcefield
for molname in ['cyclohexane', 'ethanol']:
    ifs = oechem.oemolistream(molname+'.mol2')
    ifs.SetFlavor( oechem.OEFormat_MOL2, flavor)
    mol = oechem.OEGraphMol()
    while oechem.OEReadMolecule(ifs, mol):
        oechem.OETriposAtomNames(mol)
        mols.append(oechem.OEGraphMol(mol))

system = ff.createSystem( pdbfile.topology, mols)
#Swap in positions
integrator = openmm.VerletIntegrator(1.0 * unit.femtoseconds)
context = openmm.Context(system, integrator)
context.setPositions(pdbfile.positions)

# Create parmed Structure
structure = parmed.openmm.topsystem.load_topology( pdbfile.topology, system, pdbfile.positions)
# Write AMBER parameter/crd
#parmed.amber.AmberParm( system, structure)
#help(structure.save)
#Save prmtop
structure.save('system.prmtop', overwrite=True)
# Save crd
structure.save('system.crd', format='rst7', overwrite=True)

{0: -0.07609999924898148, 1: -0.0754999965429306, 2: -0.0754999965429306, 3: -0.0754999965429306, 4: -0.0754999965429306, 5: -0.0754999965429306, 6: 0.03779999911785126, 7: 0.03779999911785126, 8: 0.03779999911785126, 9: 0.03779999911785126, 10: 0.03779999911785126, 11: 0.03779999911785126, 12: 0.03779999911785126, 13: 0.03779999911785126, 14: 0.03779999911785126, 15: 0.03779999911785126, 16: 0.03779999911785126, 17: 0.03779999911785126}
{0: -0.09669999778270721, 1: 0.12970000505447388, 2: -0.5995000004768372, 3: 0.04479999840259552, 4: 0.04479999840259552, 5: 0.04479999840259552, 6: 0.017100000753998756, 7: 0.017100000753998756, 8: 0.3978999853134155}


In [21]:
# Evaluate energy of OpenMM system and AMBER system to check that they are still the same
from smarty.forcefield_utils import *
parm = parmed.load_file('system.prmtop', 'system.crd')
ambersys = parm.createSystem(nonbondedMethod= app.NoCutoff, constraints = None, implicitSolvent = None)
groups0, groups1, energy0, energy1 = compare_system_energies( pdbfile.topology, pdbfile.topology, ambersys, system, pdbfile.positions)

# Load as in create_system_from_amber; this fails. Apparently some sort of OpenMM issue.
#prmtop = app.AmberPrmtopFile('system.prmtop')
#system =prmtop.createSystem(nonbondedMethod = app.NoCutoff, constraints = None, implicitSolvent = None)


Num (type) 	 Num (type) 	 Num (type) 	 Num (type) 	 per 	 phase 	 k0
128 ( C3)- 127 ( C2)- 	 126 ( C1)- 	 131 ( C6)- 	 3.000000 	 0.000000 	 0.753120 
128 ( C3)- 127 ( C2)- 	 126 ( C1)- 	 131 ( C6)- 	 2.000000 	 180.000000 	 1.046000 
128 ( C3)- 127 ( C2)- 	 126 ( C1)- 	 131 ( C6)- 	 1.000000 	 180.000000 	 0.836800 
1009 ( C2)- 1010 ( C3)- 	 1011 ( C4)- 	 1012 ( C5)- 	 3.000000 	 0.000000 	 0.753120 
1009 ( C2)- 1010 ( C3)- 	 1011 ( C4)- 	 1012 ( C5)- 	 2.000000 	 180.000000 	 1.046000 
1009 ( C2)- 1010 ( C3)- 	 1011 ( C4)- 	 1012 ( C5)- 	 1.000000 	 180.000000 	 0.836800 
379 ( C2)- 378 ( C1)- 	 383 ( C6)- 	 382 ( C5)- 	 3.000000 	 0.000000 	 0.753120 
379 ( C2)- 378 ( C1)- 	 383 ( C6)- 	 382 ( C5)- 	 2.000000 	 180.000000 	 1.046000 
379 ( C2)- 378 ( C1)- 	 383 ( C6)- 	 382 ( C5)- 	 1.000000 	 180.000000 	 0.836800 
470 ( C3)- 469 ( C2)- 	 468 ( C1)- 	 473 ( C6)- 	 3.000000 	 0.000000 	 0.753120 
470 ( C3)- 469 ( C2)- 	 468 ( C1)- 	 473 ( C6)- 	 2.000000 	 180.000000 	 1.046000 
470

AssertionError: Error: NonBondedForce Exception atom (2702, 2705) has charge values of -0.026858 and -0.022381 elementary charge, respectively.

In [12]:
product = structure.atoms[2702].charge*structure.atoms[2705].charge
print(product)
print(parm.atoms[2702].charge*parm.atoms[2705].charge)
print(structure.atoms[2702].name)
print(structure.atoms[2705].name)
print(structure.atoms[2702].charge)
print(structure.atoms[2705].charge)

# By what factor do they differ?
fac = 0.026858/0.022381
print(fac)
1/1.2

-0.0268575990637
-0.0268575991688
O1
H3
-0.599500000477
0.0447999984026
1.2000357446


0.8333333333333334

In [16]:
# Dump OpenMM systems out

# Do smarty system
from simtk.openmm import XmlSerializer
serialized_system_smirff = XmlSerializer.serialize(system)
outfile = open('smarty_system.xml','w')
outfile.write(serialized_system_smirff)
outfile.close()
# Do amber system
serialized_system_amber = XmlSerializer.serialize(ambersys)
outfile = open('amber_system.xml','w')
outfile.write(serialized_system_amber)
outfile.close()

In [22]:
# deserialize systems and confirm energies still different without using external tools
smirff_system = XmlSerializer.deserialize(serialized_system_smirff)
amber_system = XmlSerializer.deserialize(serialized_system_amber)
pdbfile = PDBFile('cyclohexane_ethanol_0.4_0.6.pdb')

# Do an energy comparison
timestep = 1.0 * unit.femtoseconds
integrator0 = simtk.openmm.VerletIntegrator( timestep )
integrator1 = simtk.openmm.VerletIntegrator( timestep )
platform = simtk.openmm.Platform.getPlatformByName("Reference")
positions = pdbfile.positions
simulation0 = app.Simulation( pdbfile.topology, smirff_system, integrator0, platform = platform )
simulation0.context.setPositions(positions)
simulation1 = app.Simulation( parm.topology, amber_system, integrator1, platform = platform )
simulation1.context.setPositions(positions)
syscheck = system_checker.SystemChecker( simulation0, simulation1 )
groups0, groups1 = syscheck.check_energy_groups(skip_assert = False)
energy0, energy1 = syscheck.check_energies(skip_assert = False)

AssertionError: Error, NB energy difference (3666.980778 kJ/mol) is greater than 12.552000 kJ/mol